In [5]:
#Import some libraries
import tensorflow as tf
import glob
import numpy as np
# for installer: pip install Pillow
from PIL import Image
import matplotlib.pyplot as plt
from skimage import transform
import os

In [12]:
width = 68
height = 68
X = []
y = []
def get_dataset(path_image, Cat=True):
    images = glob.glob(path_image+"*.png")
    
    for image in images:
        img = Image.open(image)
        img = np.asarray(img)
        if Cat:
            y.append(1)
        else:
            y.append(0)
        img = transform.resize(img,(width, height))
        X.append(img)
        
path_images_cat = 'train/cat/'
path_images_n_cat = 'train/noncat/'

get_dataset(path_images_cat)
get_dataset(path_images_n_cat, False)

X = np.asarray(X)
y = np.asarray(y)

In [13]:
# check the shape of data set and labels
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (209, 68, 68, 3)
Shape of y: (209,)


In [20]:
epochs = 500
display_step = 100
batch = 64
shape = X.shape[1]*X.shape[2]*X.shape[3]

In [21]:
# The first dimension (None) will index the 
# images in the batch
X_ = tf.placeholder(tf.float32, [None, width, height, 3])
# The correct answers will go here
y_ = tf.placeholder(tf.float32, [None, 2])

# Build the matrix weights[13872, 2]
# initialized with random values from normal
# distribution mean = 0, stddev=0.1
W = tf.Variable(
    tf.truncated_normal([shape, 2], 
                        stddev=0.1))

# biase b[2]
b = tf.Variable(tf.zeros([2]))


In [22]:
# Flatten the images, 
# unroll each image row by row
# create vector[13872]
XX = tf.reshape(X_, [-1, shape])

#Define model
# Y = tf.nn.